# Practice 08 — Tensor Manipulation

Original Authors: Seungjae Ryan Lee, Ki Hyun Kim  
Source: https://github.com/deeplearningzerotoall/PyTorch

Practice 07에서 Tensor 생성과 Autograd를 배웠습니다.  
이 노트북에서는 PyTorch Tensor의 **조작(manipulation)** 연산을 익힙니다.

| 핵심 개념 | 설명 |
|---|---|
| `view` / `reshape` | 텐서 형태 변환 (NumPy의 `reshape`에 해당) |
| `matmul` / `@` | 행렬 곱셈 |
| `squeeze` / `unsqueeze` | 차원 축소/확장 |
| `cat` / `stack` | 텐서 결합 |

In [ ]:
import numpy as np
import torch

---
# 1. NumPy → PyTorch

PyTorch 텐서는 NumPy 배열과 거의 동일한 API를 가지고 있습니다.  
NumPy에서 익숙한 연산들이 PyTorch에서도 같은 방식으로 동작합니다.

### 1D Tensor

In [ ]:
t = torch.FloatTensor([0., 1., 2., 3., 4., 5., 6.])
print(t)
print(f'dim : {t.dim()}')     # rank (차원 수)
print(f'shape: {t.shape}')    # shape
print(f'size : {t.size()}')   # shape과 동일

In [ ]:
# 인덱싱과 슬라이싱 — NumPy와 동일
print('t[0], t[1], t[-1] =', t[0].item(), t[1].item(), t[-1].item())
print('t[2:5]  =', t[2:5])     # 인덱스 2~4
print('t[:2]   =', t[:2])      # 처음~1
print('t[3:]   =', t[3:])      # 3~끝

### 2D Tensor

In [ ]:
t = torch.FloatTensor([[1., 2., 3.],
                        [4., 5., 6.],
                        [7., 8., 9.],
                        [10., 11., 12.]])
print(t)
print(f'dim : {t.dim()}')      # 2
print(f'size: {t.size()}')     # (4, 3)

In [ ]:
# 2D 슬라이싱
print('2번째 열:', t[:, 1])          # 모든 행의 2번째 열
print('마지막 열 제외:', t[:, :-1])   # 모든 행에서 마지막 열 제외

### Shape, Rank, Axis

| 용어 | 의미 | 예시 |
|---|---|---|
| **Rank** | 차원 수 (`dim()`) | 2D 텐서 = rank 2 |
| **Shape** | 각 차원의 크기 | `(4, 3)` = 4행 3열 |
| **Axis** | 특정 차원 방향 | `dim=0` (행 방향), `dim=1` (열 방향) |

In [ ]:
# 고차원 텐서 — 이미지 배치를 표현할 때 사용
# (batch, channel, height, width)
t = torch.FloatTensor([[[[1, 2, 3, 4],
                          [5, 6, 7, 8],
                          [9, 10, 11, 12]],
                         [[13, 14, 15, 16],
                          [17, 18, 19, 20],
                          [21, 22, 23, 24]]]])

print(f'dim : {t.dim()}')      # rank = 4
print(f'size: {t.size()}')     # (1, 2, 3, 4) — batch=1, channel=2, H=3, W=4

### PyTorch ↔ NumPy 변환

| 메서드 | 설명 |
|---|---|
| `torch.tensor(arr)` | NumPy 배열 → 텐서 (복사) |
| `torch.from_numpy(arr)` | NumPy 배열 → 텐서 (**메모리 공유**) |
| `.numpy()` | 텐서 → NumPy 배열 (`requires_grad=True`이면 `.detach()` 먼저) |
| `.item()` | 단일 원소 텐서 → Python 숫자 |
| `.detach()` | autograd 그래프에서 분리한 새 텐서 반환 |

In [ ]:
# .item() — 단일 원소 텐서를 Python 숫자로 변환
t = torch.tensor(3.14)
print(f'item(): {t.item()},  type: {type(t.item()).__name__}')

# .detach() — autograd 그래프에서 분리
t_grad = torch.tensor(2.0, requires_grad=True)
t_detached = t_grad.detach()
print(f'\nrequires_grad: {t_grad.requires_grad}  →  detach: {t_detached.requires_grad}')

# .detach().numpy() — grad 추적 중인 Tensor → NumPy
arr = t_grad.detach().numpy()
print(f'detach().numpy(): {arr},  type: {type(arr).__name__}')

# torch.from_numpy() — NumPy → Tensor (메모리 공유: 한쪽 수정 시 다른 쪽도 변경)
np_arr = np.array([1.0, 2.0, 3.0])
t_shared = torch.from_numpy(np_arr)
print(f'\nfrom_numpy: {t_shared}')
np_arr[0] = 999.0
print(f'NumPy 수정 후 Tensor도 변경: {t_shared}')   # 메모리 공유 확인

---
# 2. 텐서 연산

### Mul vs Matmul

Practice 01에서 배운 `*` (element-wise)와 `@` (행렬 곱셈)의 차이가 PyTorch에서도 동일하게 적용됩니다.

| 연산 | 의미 | PyTorch |
|---|---|---|
| 원소별 곱셈 | $a_{ij} \times b_{ij}$ | `*`, `.mul()` |
| 행렬 곱셈 | $\sum_k a_{ik} b_{kj}$ | `@`, `.matmul()` |

In [ ]:
m1 = torch.FloatTensor([[1, 2], [3, 4]])
m2 = torch.FloatTensor([[1], [2]])
print('Shape of m1:', m1.shape)   # (2, 2)
print('Shape of m2:', m2.shape)   # (2, 1)

# 행렬 곱셈: (2,2) @ (2,1) = (2,1)
print('\nMatmul (m1 @ m2):')
print(m1.matmul(m2))

# 원소별 곱셈: broadcasting으로 (2,2) * (2,1) = (2,2)
print('\nElement-wise (m1 * m2):')
print(m1 * m2)
print(m1.mul(m2))               # 동일

### Broadcasting

크기가 다른 텐서끼리 연산할 때, PyTorch가 자동으로 크기를 맞춰줍니다.

> **주의:** Broadcasting은 편리하지만, 의도하지 않은 결과를 낼 수 있습니다.  
> shape을 항상 확인하는 습관이 중요합니다.

In [ ]:
# 같은 shape — broadcasting 없음
m1 = torch.FloatTensor([[3, 3]])
m2 = torch.FloatTensor([[2, 2]])
print('Same shape:', m1 + m2)

In [ ]:
# Vector + Scalar: [3] -> [[3, 3]] 으로 확장
m1 = torch.FloatTensor([[1, 2]])
m2 = torch.FloatTensor([3])
print('Vector + Scalar:', m1 + m2)

In [ ]:
# (1,2) + (2,1) -> (2,2)  — 양쪽 모두 확장
m1 = torch.FloatTensor([[1, 2]])       # (1, 2)
m2 = torch.FloatTensor([[3], [4]])     # (2, 1)
print(m1 + m2)
# [[1,2]] + [[3],    [[1+3, 2+3],    [[4, 5],
#            [4]]  =  [1+4, 2+4]]  =  [5, 6]]

---
# 3. 축(dim)별 집계

NumPy의 `axis`와 동일한 개념입니다.  
`dim=0`은 **행 방향(위→아래)**으로 집계, `dim=1`은 **열 방향(왼→오)**으로 집계합니다.

### Mean

In [ ]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)
print(f'\nmean()      = {t.mean()}')         # 전체 평균: 2.5
print(f'mean(dim=0) = {t.mean(dim=0)}')      # 열별 평균: [2, 3]
print(f'mean(dim=1) = {t.mean(dim=1)}')      # 행별 평균: [1.5, 3.5]

In [ ]:
# 정수 텐서는 mean() 사용 불가 — float으로 변환 필요
t_int = torch.LongTensor([1, 2])
try:
    print(t_int.mean())
except Exception as e:
    print(f'Error: {e}')
    print(f'Solution: {t_int.float().mean()}')

### Sum

In [ ]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)
print(f'\nsum()      = {t.sum()}')           # 전체 합: 10
print(f'sum(dim=0) = {t.sum(dim=0)}')        # 열별 합: [4, 6]
print(f'sum(dim=1) = {t.sum(dim=1)}')        # 행별 합: [3, 7]

### Max / Argmax

`max()`에 `dim`을 지정하면 **(최대값, 인덱스)** 두 개를 반환합니다.  
`argmax`는 분류 모델에서 **예측 클래스를 선택**할 때 핵심적으로 사용됩니다.

```python
# 이후 실습에서 자주 볼 코드:
pred = logits.argmax(dim=1)  # 각 샘플의 예측 클래스
```

In [ ]:
t = torch.FloatTensor([[1, 2], [3, 4]])
print(t)

# dim 없이 — 전체 최대값 1개
print(f'\nmax() = {t.max()}')

# dim=0 — 열별 최대값 + 해당 행 인덱스
values, indices = t.max(dim=0)
print(f'\nmax(dim=0):')
print(f'  values : {values}')     # [3, 4]
print(f'  indices: {indices}')    # [1, 1] — 둘 다 행 인덱스 1

# dim=1 — 행별 최대값 + 해당 열 인덱스
values, indices = t.max(dim=1)
print(f'\nmax(dim=1):')
print(f'  values : {values}')     # [2, 4]
print(f'  indices: {indices}')    # [1, 1] — 둘 다 열 인덱스 1

---
# 4. Shape 변환

### view — NumPy의 reshape에 해당

`view`는 텐서의 **원소 수를 유지**하면서 형태를 바꿉니다.  
`-1`을 사용하면 해당 차원의 크기를 **자동 계산**합니다.

```python
# 이후 실습에서 자주 볼 코드:
x = x.view(-1, 784)      # (N, 28, 28) -> (N, 784)  FC 입력
y = y.view(-1, 1)        # (N,) -> (N, 1)  열 벡터
```

In [ ]:
t = torch.FloatTensor([[[0, 1, 2],
                         [3, 4, 5]],
                        [[6, 7, 8],
                         [9, 10, 11]]])
print(f'Original shape: {t.shape}')    # (2, 2, 3)

# (2,2,3) -> (4,3)  — -1은 자동 계산: 12/3 = 4
print(f'\nview(-1, 3):')
print(t.view(-1, 3))
print(f'shape: {t.view(-1, 3).shape}')

# (2,2,3) -> (4,1,3)
print(f'\nview(-1, 1, 3):')
print(t.view(-1, 1, 3))
print(f'shape: {t.view(-1, 1, 3).shape}')

### squeeze — 크기 1인 차원 제거

모델 출력이 `(N, 1)` 형태일 때, `squeeze()`로 `(N,)`으로 만들 수 있습니다.

In [ ]:
ft = torch.FloatTensor([[0], [1], [2]])
print(f'Original: {ft.shape}')          # (3, 1)
print(ft)

print(f'\nSqueeze: {ft.squeeze().shape}')  # (3,)
print(ft.squeeze())

### unsqueeze — 차원 추가

`squeeze`의 반대 연산입니다. 지정한 위치에 **크기 1인 차원을 삽입**합니다.

```python
# 이후 실습에서 자주 볼 코드:
x = x.unsqueeze(0)     # (C, H, W) -> (1, C, H, W)  배치 차원 추가
```

In [ ]:
ft = torch.Tensor([0, 1, 2])
print(f'Original: {ft.shape}')               # (3,)

# dim=0: 맨 앞에 차원 추가 -> (1, 3)  행 벡터
print(f'\nunsqueeze(0): {ft.unsqueeze(0).shape}')
print(ft.unsqueeze(0))

# dim=1: 두 번째에 차원 추가 -> (3, 1)  열 벡터
print(f'\nunsqueeze(1): {ft.unsqueeze(1).shape}')
print(ft.unsqueeze(1))

# view로도 동일한 결과
print(f'\nview(1, -1): {ft.view(1, -1).shape}')   # unsqueeze(0)과 동일
print(f'view(-1, 1): {ft.view(-1, 1).shape}')     # unsqueeze(1)과 동일

---
# 5. 기타 유용한 연산

### Type Casting

텐서의 자료형을 변환합니다. 모델 입력은 보통 `float`, 레이블은 `long`을 사용합니다.

In [ ]:
lt = torch.LongTensor([1, 2, 3, 4])
print(f'LongTensor : {lt}')
print(f'.float()   : {lt.float()}')

bt = torch.BoolTensor([True, False, False, True])
print(f'\nBoolTensor : {bt}')
print(f'.long()    : {bt.long()}')
print(f'.float()   : {bt.float()}')

### Concatenation (torch.cat)

**기존 차원**을 따라 텐서를 이어 붙입니다.  
NumPy의 `np.concatenate`, `np.vstack`, `np.hstack`에 해당합니다.

In [ ]:
x = torch.FloatTensor([[1, 2], [3, 4]])
y = torch.FloatTensor([[5, 6], [7, 8]])

# dim=0: 행 방향으로 이어 붙이기 (= np.vstack)
print('cat dim=0 (vstack):')
print(torch.cat([x, y], dim=0))

# dim=1: 열 방향으로 이어 붙이기 (= np.hstack)
print('\ncat dim=1 (hstack):')
print(torch.cat([x, y], dim=1))

### Stacking (torch.stack)

**새로운 차원**을 만들어서 텐서를 쌓습니다.  
`cat`과 달리 차원이 하나 증가합니다.

In [ ]:
x = torch.FloatTensor([1, 4])
y = torch.FloatTensor([2, 5])
z = torch.FloatTensor([3, 6])

# 기본 dim=0: 각 텐서가 하나의 행이 됨
print('stack dim=0:')
print(torch.stack([x, y, z]))
print(f'shape: {torch.stack([x, y, z]).shape}')     # (3, 2)

# dim=1: 각 텐서가 하나의 열이 됨
print('\nstack dim=1:')
print(torch.stack([x, y, z], dim=1))
print(f'shape: {torch.stack([x, y, z], dim=1).shape}')  # (2, 3)

# stack = unsqueeze + cat 과 동일
print('\ncat + unsqueeze (동일 결과):')
print(torch.cat([x.unsqueeze(0), y.unsqueeze(0), z.unsqueeze(0)], dim=0))

### ones_like / zeros_like

기존 텐서와 **같은 shape, 같은 dtype**으로 1 또는 0으로 채운 텐서를 생성합니다.

In [ ]:
x = torch.FloatTensor([[0, 1, 2], [2, 1, 0]])
print('x:')
print(x)
print(f'\nones_like : {torch.ones_like(x)}')
print(f'zeros_like: {torch.zeros_like(x)}')

### In-place 연산

메서드 이름 뒤에 `_`(언더스코어)를 붙이면 **원본 텐서를 직접 수정**합니다.  
`_` 없으면 새 텐서를 반환하고 원본은 그대로입니다.

In [ ]:
x = torch.FloatTensor([[1, 2], [3, 4]])

# mul(): 새 텐서 반환, 원본 유지
print('mul(2):', x.mul(2))
print('x (unchanged):', x)

# mul_(): 원본 직접 수정 (in-place)
print('\nmul_(2):', x.mul_(2))
print('x (changed!):', x)

### One-hot Encoding (scatter)

클래스 인덱스를 **one-hot 벡터**로 변환합니다.  
예: 클래스 2 (3개 클래스) → `[0, 0, 1]`

In [ ]:
# 4개 샘플의 클래스 인덱스
labels = torch.LongTensor([[0], [1], [2], [0]])
print('Labels:', labels.flatten().tolist())

# one-hot 변환: (batch_size, num_classes) 크기의 0 텐서에 scatter
one_hot = torch.zeros(4, 3)              # 4 samples, 3 classes
one_hot.scatter_(1, labels, 1)           # dim=1, 인덱스 위치에 1.0 채움
print('\nOne-hot:')
print(one_hot)

---
# 요약

| 카테고리 | 연산 | 설명 |
|---|---|---|
| **생성** | `torch.FloatTensor`, `zeros`, `ones` | 텐서 생성 |
| **속성** | `dim()`, `shape`, `size()` | 차원, 형태 조회 |
| **연산** | `*` / `@`, `matmul` | 원소별 곱 / 행렬 곱 |
| **집계** | `mean`, `sum`, `max` | `dim` 지정으로 축별 집계 |
| **변환** | `view`, `squeeze`, `unsqueeze` | shape 변환 |
| **결합** | `cat`, `stack` | 기존 차원 / 새 차원으로 결합 |
| **기타** | `float()`, `long()`, `_` suffix | 타입 변환, in-place 연산 |

> 다음 Practice에서는 이 연산들을 활용하여 **선형 모델**을 PyTorch로 구현합니다.